In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import seaborn as sns
# Importing the required packages 
import pandas as pd 
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [2]:
KDDTrain = pd.read_csv('KDDTest+.csv')
KDDTest = pd.read_csv('KDDTrain+.csv')


In [3]:
mapping = {'ipsweep': 'Probe','satan': 'Probe','nmap': 'Probe','portsweep': 'Probe','saint': 'Probe','mscan': 'Probe',
        'teardrop': 'DoS','pod': 'DoS','land': 'DoS','back': 'DoS','neptune': 'DoS','smurf': 'DoS','mailbomb': 'DoS',
        'udpstorm': 'DoS','apache2': 'DoS','processtable': 'DoS',
        'perl': 'U2R','loadmodule': 'U2R','rootkit': 'U2R','buffer_overflow': 'U2R','xterm': 'U2R','ps': 'U2R',
        'sqlattack': 'U2R','httptunnel': 'U2R',
        'ftp_write': 'R2L','phf': 'R2L','guess_passwd': 'R2L','warezmaster': 'R2L','warezclient': 'R2L','imap': 'R2L',
        'spy': 'R2L','multihop': 'R2L','named': 'R2L','snmpguess': 'R2L','worm': 'R2L','snmpgetattack': 'R2L',
        'xsnoop': 'R2L','xlock': 'R2L','sendmail': 'R2L',
        'normal': 'Normal'
        }

In [4]:
# Apply attack class mappings to the dataset
KDDTrain['attack_class'] = KDDTrain['labels'].apply(lambda v: mapping[v])
KDDTest['attack_class'] = KDDTest['labels'].apply(lambda v: mapping[v])

In [5]:
KDDTrain

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,labels,attack_class
0,0,tcp,private,REJ,0,0,0,0,0,0,...,0.04,0.06,0.00,0.00,0.00,0.0,1.00,1.00,neptune,DoS
1,0,tcp,private,REJ,0,0,0,0,0,0,...,0.00,0.06,0.00,0.00,0.00,0.0,1.00,1.00,neptune,DoS
2,2,tcp,ftp_data,SF,12983,0,0,0,0,0,...,0.61,0.04,0.61,0.02,0.00,0.0,0.00,0.00,normal,Normal
3,0,icmp,eco_i,SF,20,0,0,0,0,0,...,1.00,0.00,1.00,0.28,0.00,0.0,0.00,0.00,saint,Probe
4,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,0.31,0.17,0.03,0.02,0.00,0.0,0.83,0.71,mscan,Probe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22539,0,tcp,smtp,SF,794,333,0,0,0,0,...,0.72,0.06,0.01,0.01,0.01,0.0,0.00,0.00,normal,Normal
22540,0,tcp,http,SF,317,938,0,0,0,0,...,1.00,0.00,0.01,0.01,0.01,0.0,0.00,0.00,normal,Normal
22541,0,tcp,http,SF,54540,8314,0,0,0,2,...,1.00,0.00,0.00,0.00,0.00,0.0,0.07,0.07,back,DoS
22542,0,udp,domain_u,SF,42,42,0,0,0,0,...,0.99,0.01,0.00,0.00,0.00,0.0,0.00,0.00,normal,Normal


## Random Forest 

In [6]:
from sklearn import preprocessing
from scipy.io.arff import loadarff
from sklearn.ensemble import RandomForestClassifier
import numpy as np

In [7]:
# Preprocess
enc = preprocessing.OrdinalEncoder()

encoded_dataset = enc.fit_transform(KDDTrain)  # All categorical features are now numerical
X_train = encoded_dataset[:, :-2]  # All rows, omit last column
y_train = np.ravel(encoded_dataset[:, -1:])  # All rows, only the last column

In [8]:
# Repeat preprocessing for test data
encoded_dataset = enc.fit_transform(KDDTest)
X_test = encoded_dataset[:, :-2]
y_test = np.ravel(encoded_dataset[:, -1:])


In [9]:
# Fit to model and predict
rfc = RandomForestClassifier()
y_pred = rfc.fit(X_train, y_train).predict(X_test)

In [10]:
total_datapoints = X_test.shape[0]
mislabeled_datapoints = (y_test != y_pred).sum()
correct_datapoints = total_datapoints-mislabeled_datapoints
percent_correct = (correct_datapoints / total_datapoints) * 100

print("RandomForestClassifier results for NSL-KDD:\n")
print("Total datapoints: %d\nCorrect datapoints: %d\nMislabeled datapoints: %d\nPercent correct: %.2f%%"
      % (total_datapoints, correct_datapoints, mislabeled_datapoints, percent_correct))

RandomForestClassifier results for NSL-KDD:

Total datapoints: 125973
Correct datapoints: 117550
Mislabeled datapoints: 8423
Percent correct: 93.31%


In [11]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred))

[[44583  1248    96     0     0]
 [  251 62922  2356  1750    64]
 [  140  1632  9862    21     1]
 [    4   819     3   163     6]
 [    1    25     0     6    20]]
              precision    recall  f1-score   support

         0.0       0.99      0.97      0.98     45927
         1.0       0.94      0.93      0.94     67343
         2.0       0.80      0.85      0.82     11656
         3.0       0.08      0.16      0.11       995
         4.0       0.22      0.38      0.28        52

    accuracy                           0.93    125973
   macro avg       0.61      0.66      0.63    125973
weighted avg       0.94      0.93      0.94    125973

0.9331364657505974


In [12]:
KDDTrain['attack_class'].unique()

array(['DoS', 'Normal', 'Probe', 'R2L', 'U2R'], dtype=object)

####  Label Encoding

In [13]:
KDDTrain

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,labels,attack_class
0,0,tcp,private,REJ,0,0,0,0,0,0,...,0.04,0.06,0.00,0.00,0.00,0.0,1.00,1.00,neptune,DoS
1,0,tcp,private,REJ,0,0,0,0,0,0,...,0.00,0.06,0.00,0.00,0.00,0.0,1.00,1.00,neptune,DoS
2,2,tcp,ftp_data,SF,12983,0,0,0,0,0,...,0.61,0.04,0.61,0.02,0.00,0.0,0.00,0.00,normal,Normal
3,0,icmp,eco_i,SF,20,0,0,0,0,0,...,1.00,0.00,1.00,0.28,0.00,0.0,0.00,0.00,saint,Probe
4,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,0.31,0.17,0.03,0.02,0.00,0.0,0.83,0.71,mscan,Probe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22539,0,tcp,smtp,SF,794,333,0,0,0,0,...,0.72,0.06,0.01,0.01,0.01,0.0,0.00,0.00,normal,Normal
22540,0,tcp,http,SF,317,938,0,0,0,0,...,1.00,0.00,0.01,0.01,0.01,0.0,0.00,0.00,normal,Normal
22541,0,tcp,http,SF,54540,8314,0,0,0,2,...,1.00,0.00,0.00,0.00,0.00,0.0,0.07,0.07,back,DoS
22542,0,udp,domain_u,SF,42,42,0,0,0,0,...,0.99,0.01,0.00,0.00,0.00,0.0,0.00,0.00,normal,Normal


In [14]:
from sklearn.preprocessing import LabelEncoder
le_protocol = LabelEncoder()
le_service = LabelEncoder()
le_flag = LabelEncoder()
KDDTrain['protocol_type'] = le_protocol.fit_transform(KDDTrain.protocol_type)
KDDTrain['service'] = le_protocol.fit_transform(KDDTrain.service)
KDDTrain['flag'] = le_protocol.fit_transform(KDDTrain.flag)
KDDTrain
KDDTest['protocol_type'] = le_protocol.fit_transform(KDDTest.protocol_type)
KDDTest['service'] = le_protocol.fit_transform(KDDTest.service)
KDDTest['flag'] = le_protocol.fit_transform(KDDTest.flag)


#### Predicting labels

In [15]:
X_train = KDDTrain.iloc[:,0:40]  # All rows, omit last 2 columns
y_train = KDDTrain.iloc[:,41]# All rows, only the second last column

X_test = KDDTest.iloc[:,0:40]  # All rows, omit last 2 column
y_test = KDDTest.iloc[:,41]# All rows, only the second last  column

In [16]:
from xgboost import XGBClassifier
model = XGBClassifier(silent=0,
                     scale_pos_weight=1,
                     learning_rate=0.01,
                     colsample_bytree=0.6,
                     subsample=0.8,
                     objective='binary:logistic',
                     n_estimators=100,
                     reg_alpha=0.3,
                     max_depth=3,
                     gamma=1)

model.fit(X_train,y_train,verbose=True)


[13:11:19] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { scale_pos_weight, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6, gamma=1, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.01, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0.3,
              reg_lambda=1, scale_pos_weight=1, silent=0, subsample=0.8,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [17]:
preds = model.predict(X_test)

print(accuracy_score(y_test,preds))

0.9094409119414477


In [18]:
import xgboost as xgb
boost = xgb.XGBClassifier(max_depth=9,
                          subsample=0.9,
                          objective='multi:softmax',
                          num_class = 3,
                          min_child_weight=2,
                          colsample_bytree=0.7,
                          n_estimators=1000,
                          learning_rate=0.08,
                          n_jobs = -1)
boost.fit(X_train,y_train)
boost_pred = boost.predict(X_test)
accuracy_score(y_test,boost_pred)

0.9153786922594525

In [19]:


print(confusion_matrix(y_test,boost_pred))  
print(classification_report(y_test,boost_pred))  
print(accuracy_score(y_test, boost_pred))

[[  0   0   0 ...   0   0   0]
 [  1 948   0 ...   0   0   0]
 [  0   0   6 ...   0   0   8]
 ...
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]]


C:\Users\ninadgorey\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ninadgorey\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                 precision    recall  f1-score   support

        apache2       0.00      0.00      0.00         0
           back       1.00      0.99      1.00       956
buffer_overflow       0.11      0.20      0.14        30
      ftp_write       0.00      0.00      0.00         8
   guess_passwd       0.51      0.98      0.67        53
     httptunnel       0.00      0.00      0.00         0
           imap       0.00      0.00      0.00        11
        ipsweep       0.97      0.81      0.88      3599
           land       0.67      0.56      0.61        18
     loadmodule       0.00      0.00      0.00         9
       mailbomb       0.00      0.00      0.00         0
          mscan       0.00      0.00      0.00         0
       multihop       0.00      0.00      0.00         7
          named       0.00      0.00      0.00         0
        neptune       1.00      1.00      1.00     41214
           nmap       0.98      0.18      0.30      1493
         normal       0.93    

### Predicting Class attack

In [20]:
X_train = KDDTrain.iloc[:,0:40]  # All rows, omit last  2 column
y_train = KDDTrain.iloc[:,42]# All rows, only the last column

X_test = KDDTest.iloc[:,0:40]  # All rows, omit last 2 column
y_test = KDDTest.iloc[:,42]# All rows, only the last column

In [21]:
from xgboost import XGBClassifier
model = XGBClassifier(silent=0,
                     scale_pos_weight=1,
                     learning_rate=0.01,
                     colsample_bytree=0.6,
                     subsample=0.8,
                     objective='binary:logistic',
                     n_estimators=100,
                     reg_alpha=0.3,
                     max_depth=3,
                     gamma=1)

model.fit(X_train,y_train,verbose=True)


[13:23:51] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { scale_pos_weight, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6, gamma=1, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.01, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0.3,
              reg_lambda=1, scale_pos_weight=1, silent=0, subsample=0.8,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [22]:
preds = model.predict(X_test)

print(accuracy_score(y_test,preds))

0.9104411262730903


In [23]:
import xgboost as xgb
boost = xgb.XGBClassifier(max_depth=9,
                          subsample=0.9,
                          objective='multi:softmax',
                          num_class = 3,
                          min_child_weight=2,
                          colsample_bytree=0.7,
                          n_estimators=1000,
                          learning_rate=0.08,
                          n_jobs = -1)
boost.fit(X_train,y_train)
boost_pred = boost.predict(X_test)

In [24]:
accuracy_score(y_test,boost_pred)

0.9376136156160447

In [25]:


print(confusion_matrix(y_test,boost_pred))  
print(classification_report(y_test,boost_pred))  
print(accuracy_score(y_test, boost_pred))

[[44301  1567    53     1     5]
 [  449 63792  1918  1089    95]
 [  100  1686  9851    18     1]
 [    3   831     5   134    22]
 [    1    12     0     3    36]]
              precision    recall  f1-score   support

         DoS       0.99      0.96      0.98     45927
      Normal       0.94      0.95      0.94     67343
       Probe       0.83      0.85      0.84     11656
         R2L       0.11      0.13      0.12       995
         U2R       0.23      0.69      0.34        52

    accuracy                           0.94    125973
   macro avg       0.62      0.72      0.64    125973
weighted avg       0.94      0.94      0.94    125973

0.9376136156160447
